```
conda create --name deepchem python=3.9
conda activate deepchem

<!-- conda install -c conda-forge -c deepchem deepchem -->
conda install -c conda-forge rdkit
pip uninstall deepchem && pip install --pre deepchem
conda install tensorflow
conda install pytorch torchvision -c pytorch
```

In [1]:
import deepchem as dc
import tensorflow as tf
import numpy as np
import torch

print(dc.__version__)
print(tf.__version__)
print(torch.__version__)
# print(np.__version__)

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/opt/homebrew/anaconda3/envs/deepchem/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


2.7.2.dev
2.11.0
1.13.1


In [2]:
x = np.random.random((4, 5))
y = np.random.random((4, 1))
print(x, y)

[[0.26937458 0.8374568  0.33205136 0.91139189 0.23770597]
 [0.18091067 0.17405365 0.28883541 0.26053203 0.02116718]
 [0.3052232  0.75548681 0.16088135 0.13267427 0.44419444]
 [0.9115336  0.17094345 0.88104606 0.5728637  0.86896966]] [[0.76376282]
 [0.60185755]
 [0.46477477]
 [0.34497434]]


In [3]:
dataset = dc.data.NumpyDataset(x, y)
print(dataset)

<NumpyDataset X.shape: (4, 5), y.shape: (4, 1), w.shape: (4, 1), ids: [0 1 2 3], task_names: [0]>


In [4]:
print(np.array_equal(x, dataset.X))
print(np.array_equal(y, dataset.y))

True
True


In [5]:
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()

In [6]:
print(tox21_tasks)
print(tox21_datasets)
print(transformers)

['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']
(<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>, <DiskDataset X.shape: (783, 1024), y.shape: (783, 12), w.shape: (783, 12), ids: ['N#C[C@@H]1CC(F)(F)CN1C(=O)CNC1CC2CCC(C1)N2c1ncccn1'
 'CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
 'CSc1nnc(C(C)(C)C)c(=O)n1N' ...
 'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
 'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
 'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>, <DiskDataset X.shape: (784, 1024), y.shape: (784, 12), w.shape: (784, 12), ids: ['CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O

In [7]:
train_dataset, valid_dataset, test_dataset = tox21_datasets
print(len(tox21_datasets))
print(train_dataset.X.shape)
print(train_dataset.y.shape)
print(np.shape(valid_dataset.y))

3
(6264, 1024)
(6264, 12)
(783, 12)


In [15]:
print(train_dataset.X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [8]:
print(train_dataset.w.shape)
print(np.count_nonzero(train_dataset.w))
print(np.count_nonzero(train_dataset.w == 0))

(6264, 12)
63647
11521


In [9]:
model = dc.models.RobustMultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
print(model)

In [10]:
model.fit(train_dataset, nb_epoch=10)

2023-03-01 22:44:12.932498: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


0.4318135579427083

In [11]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

In [12]:
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

In [13]:
print(train_scores)
print(test_scores)

{'mean-roc_auc_score': 0.9658544443878104}
{'mean-roc_auc_score': 0.6758106967305274}
